# I. Import & config

### Pour sauver le notebook on rajoute un if avant nos scripts pour s'assurer qu'ils ne seront exécutés que lorsqu'on les lance directement et pas dans les imports !
```python
if __name__ = '__main__':
    etc...
```

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.3'
}

# II. Get Data

In [23]:
baseUrl='https://www.numbeo.com/pollution/'

In [24]:
response = requests.get(baseUrl, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# III. Main

In [25]:
h1_text = soup.h1.text
print(h1_text)

 Pollution


In [26]:
countries = soup.select('table.related_links a')
print(countries[0]['href'])

country_result.jsp?country=Afghanistan


In [27]:
data = []

In [29]:
for country in countries:
    url = baseUrl + country['href']
    data.append(url)

In [30]:
res = pd.DataFrame(data, columns=['url'])

# IV. Store Data

In [31]:
dir_path = '../data/bronze'
os.makedirs(dir_path, exist_ok=True)

In [32]:
res.to_csv(os.path.join(dir_path, 'Pollution_urls.csv'), index=False)

# V. Iterate for each country

## Params for iteration

In [47]:
csv_file = '../data/bronze/Pollution_urls.csv'
df_urls = pd.read_csv(csv_file)
data = []

## itération

In [48]:
for url in df_urls['url']:
    print(url)

https://www.numbeo.com/pollution/country_result.jsp?country=Afghanistan
https://www.numbeo.com/pollution/country_result.jsp?country=Aland+Islands
https://www.numbeo.com/pollution/country_result.jsp?country=Albania
https://www.numbeo.com/pollution/country_result.jsp?country=Alderney
https://www.numbeo.com/pollution/country_result.jsp?country=Algeria
https://www.numbeo.com/pollution/country_result.jsp?country=American+Samoa
https://www.numbeo.com/pollution/country_result.jsp?country=Andorra
https://www.numbeo.com/pollution/country_result.jsp?country=Angola
https://www.numbeo.com/pollution/country_result.jsp?country=Anguilla
https://www.numbeo.com/pollution/country_result.jsp?country=Antigua+And+Barbuda
https://www.numbeo.com/pollution/country_result.jsp?country=Argentina
https://www.numbeo.com/pollution/country_result.jsp?country=Armenia
https://www.numbeo.com/pollution/country_result.jsp?country=Aruba
https://www.numbeo.com/pollution/country_result.jsp?country=Australia
https://www.numb

In [49]:
for url in df_urls['url']:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.content, 'html.parser')

            # Récupération du pays
            h1_text = soup.h1.text if soup.h1 else ""
            print(h1_text)
            match = re.search(r'in\s([A-Za-z\s]+)', h1_text)
            pays = match.group(1).strip().replace(' ', '_') if match else "Unknown"

            # Récupération des données de la table
            table = soup.select_one('table.table_builder_with_value_explanation.data_wide_table')
            if table:
                rows = table.select('tr')
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) > 2:
                        data.append([pays, cells[0].text.strip(), cells[2].text.strip(), cells[3].text.strip()])
        except Exception as e:
            print(f"Error processing URL {url}: {e}")

 Pollution in Afghanistan
 Pollution in Aland Islands
 Pollution in Albania
 Pollution in Alderney
 Pollution in Algeria
 Pollution in American Samoa
 Pollution in Andorra
 Pollution in Angola
 Pollution in Anguilla
 Pollution in Antigua And Barbuda
 Pollution in Argentina
 Pollution in Armenia
 Pollution in Aruba
 Pollution in Australia
 Pollution in Austria
 Pollution in Azerbaijan
 Pollution in Bahamas
 Pollution in Bahrain
 Pollution in Bangladesh
 Pollution in Barbados
 Pollution in Belarus
 Pollution in Belgium
 Pollution in Belize
 Pollution in Benin
 Pollution in Bermuda
 Pollution in Bhutan
 Pollution in Bolivia
 Pollution in Bonaire
 Pollution in Bosnia And Herzegovina
 Pollution in Botswana
 Pollution in Brazil
 Pollution in British Virgin Islands
 Pollution in Brunei
 Pollution in Bulgaria
 Pollution in Burkina Faso
 Pollution in Burundi
 Pollution in Cambodia
 Pollution in Cameroon
 Pollution in Canada
 Pollution in Cape Verde
 Pollution in Cayman Islands
 Pollution in Cen

In [50]:
data

[['Afghanistan', 'Air Pollution', '77.23\nHigh', 'High'],
 ['Afghanistan',
  'Drinking Water Pollution and Inaccessibility',
  '61.96\nHigh',
  'High'],
 ['Afghanistan',
  'Dissatisfaction with Garbage Disposal',
  '73.84\nHigh',
  'High'],
 ['Afghanistan', 'Dirty and Untidy', '68.18\nHigh', 'High'],
 ['Afghanistan', 'Noise and Light Pollution', '56.52\nModerate', 'Moderate'],
 ['Afghanistan', 'Water Pollution', '73.81\nHigh', 'High'],
 ['Afghanistan',
  'Dissatisfaction with Spending Time in the City',
  '75.00\nHigh',
  'High'],
 ['Afghanistan',
  'Dissatisfaction with Green and Parks in the City',
  '63.69\nHigh',
  'High'],
 ['Aland_Islands', 'Air Pollution', '16.67\nVery Low', 'Very Low'],
 ['Aland_Islands',
  'Drinking Water Pollution and Inaccessibility',
  '4.17\nVery Low',
  'Very Low'],
 ['Aland_Islands',
  'Dissatisfaction with Garbage Disposal',
  '12.50\nVery Low',
  'Very Low'],
 ['Aland_Islands', 'Dirty and Untidy', '16.67\nVery Low', 'Very Low'],
 ['Aland_Islands', 'Noi

# VI. Sauvegarde des Données

In [51]:
res = pd.DataFrame(data, columns=['Country', 'Pollution_Label', 'Pollution_Index', 'Pollution_Qualifier'])
res.to_csv(os.path.join(dir_path, 'Pollution.csv'), index=False)